In [152]:
import pandas as pd
import numpy as np
import scipy as sp
#import folium
import re
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns
import time

#from google.cloud import translate
#import pycountry
#import emoji

#translate_client = translate.Client()

import sys # for printing process
import unidecode # for normalizing text
from pathlib import Path # check files


from py_translator import Translator
translator = Translator()

# from googletrans import Translator

In [153]:
data_folder = './data/'
maps_folder = './maps/'

In [154]:
# CONSTANTS
# unknown values to use
UNKNOWN_NR='-1'
UNKNOWN_STR='unknown'
# delay between translation requests
TRANSLATION_DELAY=0.3
# progress in function
PROGRESS=0


In [186]:
# removes special character, numbers, accents, sets to lower
# case and removes trailing spaces
def normalizeString(string):
    cleaned=''.join([i for i in string if (i.isalnum() & ~i.isdigit()) | i.isspace() ])
    return unidecode.unidecode(cleaned.lower().strip())

def showProgress(size):
    global PROGRESS 
    PROGRESS += 1
    progress_value = int(10000*PROGRESS/size)/100
    if (progress_value*100)%1==0:
        sys.stdout.write('\r'+'Progress {}%'.format(progress_value))
        sys.stdout.flush()
def removeDuplcates(array):
    newItems=[]
    for item in array:
        if item and item not in newItems:
            newItems.append(item)
    
    return newItems

In [156]:
filename = 'en.openfoodfacts.org.products.csv'
countryfile = 'wikipedia-iso-country-codes.csv'
translationsfile = 'translations.csv'
foodfile = 'food.csv'

In [157]:
food_df = pd.read_csv(data_folder + filename, 
                      sep='\t',
                      header=0,
                      quotechar='"', 
                      low_memory=False)

In [178]:
desired_columns=[
    'ingredients_text'
]
result_column='ingredients_text'
ingredients_df=food_df.head(120).copy()

In [159]:
test=ingredients_df[ingredients_df[result_column].notna()][result_column].loc[65]
test_=test.split(',')
test_

['Rolled oats',
 ' grape concentrate',
 ' expeller pressed canola oil',
 ' sunflower seeds',
 ' almonds',
 ' walnuts oat bran',
 ' sesame seeds',
 ' cashews',
 ' natural vitamin e.']

In [150]:
FOUND_MODULE = result_column in df.columns

True

In [160]:
def removeCommonWords(value):
    words=['long','grain','white','refined','concentrate','natural','dry','roasted','organic','bar','whole','rolled','seasoning','juice','extract']
    value_=value
    for word in words:
        if word in value_:
            value_=value_.replace(word,"")
    return normalizeString(value_)

def cleanIngredients(row):
    showProgress(ingredients_df.shape[0])
    if type(row) is not list and pd.notnull(row) :
        values=row.split(',')
        ingredients=[]
        for item in values:
            ingredient=item
            # format key:value
            # take only value
            if (':') in ingredient:
                info_ = ingredient.split(':')
                if len(info_) == 2:
                    ingredient=info_[1]
            # format value (info)
            # take only values
            if ('(') in ingredient:
                info_ = ingredient.split('(')
                ingredient=info_[0]
                
            # format value [value,...]
            # take all values
            if ('[') in ingredient:
                info_ = ingredient.split('[')
                if len(info_) >= 2:
                    ingredients.append(removeCommonWords(normalizeString(info_[0])))
                    ingredient=info_[1]
            # format value [value,...]
            # take all values
            if (' or ') in ingredient:
                info_ = ingredient.split('or')
                if len(info_) >= 2:
                    ingredients.append(removeCommonWords(normalizeString(info_[0])))
                    ingredient=info_[1]
            # format value - info
            # take only values
            if (' - ') in ingredient:
                info_ = ingredient.split('-')
                ingredient=info_[0]
                
            # avoid empty strings and removeCommonWords
            ingredient=removeCommonWords(normalizeString(ingredient))
            if ingredient:
                ingredients.append(ingredient)
        
        # remove duplicate elements
        ingredients_=removeDuplcates(ingredients)
        return ingredients_
    else:
        return row

In [169]:
PROGRESS=0
cleanIngredients(test)

0.01
Progress 1.0%

['oats',
 'grape',
 'expeller pressed canola oil',
 'sunflower seeds',
 'almonds',
 'walnuts oat bran',
 'sesame seeds',
 'cashews',
 'vitamin e']

In [113]:
test

'Organic rolled oats, flame raisins, organic coconut chips, organic maple syrup, sunflower oil, walnuts, roasted diced almonds, wheat germ, honey, crunchy almond butter (dry roasted almonds), hazelnuts, almonds, organic ground cinnamon, organic vanilla extract.'

In [189]:
PROGRESS=0
ingredients_df[result_column] = ingredients_df[result_column].apply(
    lambda x: cleanIngredients(x)
)

Progress 100.0%

In [142]:
ingredients_df[ingredients_df[result_column].notna()][result_column]

10    [erythorbate de sodium, caramel, tomate, mayon...
15    [lait entier, sucre, amidon de mais, cacao, ag...
22    [baguette poite vin pain baguette, fqrine de b...
31    [paln suedois, farine de ble, eau, farine de s...
33    [taboule, legumes, huile de colza, sel, menthe...
34    [bananas, vegetable oil, corn oil andor palm o...
35    [peanuts, wheat flour, sugar, rice flour, tapi...
39    [fromage blanc  mg, creme de marron vanillee, ...
44    [pain aux  cereqles, farine, gluten de ble, eu...
46    [baguette bressan pain baguette, fqrine de ble...
63    [hazelnuts, cashews, walnuts almonds, sunflowe...
64                                            [polenta]
65    [oats, grape, expeller pressed canola oil, sun...
66                                               [rice]
67    [org oats, org hemp granola, evaporated cane, ...
68    [chocolate liquor, raw cane sugar, cocoa butte...
69         [expeller pressed, high oleic sunflower oil]
70                                       [adzuki